# PythonからGPIOポートへのアクセス

## 参考
* https://qiita.com/maoutokagura/items/9aef5e23167ce2bc1d10
* https://qiita.com/koooooo/items/e5fa77f98e6ce5c05776

* RPi.GPIO Python Module(Wiki)
    * https://sourceforge.net/p/raspberry-gpio-python/wiki/Home/
    * RPi.GPIOモジュール開発元のWikiページ
    * RPi で RPi.GPIO を使う場合，以下のパッケージをaptでインストールしておく
        * python3-rpi.gpio (Python3)
        * python-rpi.gpio  (Python2)
* Inputs
    * https://sourceforge.net/p/raspberry-gpio-python/wiki/Inputs/

## 回路の準備

* 使用ピン
    * 1ピン：3.3v電源
    * 6ピン：GND
    * 8ピン：GPIO14
    * 10ピン：GPIO15

<img src="image/circuit-1.jpg">
<img src="image/circuit-2.jpg">
<img src="image/circuit-3.jpg">


In [ ]:
# ターミナルからピン番号や状態を確認
!gpio readall

## プッシュボタンを使う

## ポーリングで読み出し

In [ ]:
# 連続的にピンの状態を確認する
# 終了するときは停止ボタンで止める

import RPi.GPIO as GPIO  # GPIOライブラリ
import time
import sys

# ピンの設定
GPIO.setwarnings(False)  # 警告を無視
GPIO.setmode(GPIO.BOARD) # ピンの番号は物理ピンの番号を使う
GPIO.setup(10, GPIO.IN, pull_up_down=GPIO.PUD_DOWN) # 10番ピンをプルダウンかつ入力に設定
GPIO.setup(8,GPIO.OUT, initial=GPIO.HIGH)    # ピン番号8番を出力に設定する．初期値はHIGH

# ピンの初期設定

try:
    while True:
        if GPIO.input(10) == GPIO.HIGH:
            GPIO.output(8, GPIO.LOW)
            print("Button was pushed!")
            time.sleep(1)
            GPIO.output(8, GPIO.HIGH)
except KeyboardInterrupt:    # 停止時の動作
        print('Interrupted')
finally:
        # GPIO設定を初期化
        GPIO.cleanup()


## イベント駆動での読み出し

* 常に入力をチェックするのではなく，押したタイミングだけ動くようにする

In [ ]:
!cat gpio_event.py

In [ ]:
!python3 gpio_event.py

In [ ]:
import RPi.GPIO as GPIO
import time
from threading import Timer

# ピンの設定
GPIO.setwarnings(False)  # 警告を無視
GPIO.setmode(GPIO.BOARD) # ピンの番号は物理ピンの番号を使う
GPIO.setup(10, GPIO.IN, pull_up_down=GPIO.PUD_DOWN) # 10番ピンをプルダウンかつ入力に設定
GPIO.setup(8,GPIO.OUT, initial=GPIO.HIGH)    # ピン番号8番を出力に設定する．初期値はHIGH

# LEDをOFFにする処理
def led_off():
    GPIO.output(8, GPIO.HIGH)

# ボタンが押されたときに実行する関数
def button_callback(channel):
    if(channel==10):
        print("Button was pushed!")
        GPIO.output(8, GPIO.LOW)
        # 1秒後にLEDをOFF
        Timer(1, led_off).start()

# 入力ピンの信号の立ち上がりでイベントを発生させる
GPIO.add_event_detect(10,GPIO.RISING,callback=button_callback, bouncetime=1000)

try:
    # キーボード入力待ち
    message = input("Press enter to quit\n\n") 
except KeyboardInterrupt:
    print('Keyboard Interrupted')
finally:
    # GPIO設定を初期化
    GPIO.cleanup()